In [121]:
from gurobipy import Model, GRB
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt

stocks = pd.read_csv("stocks.csv")

# Get all the stock names to iterate over
stock_names = stocks.columns[1:]

for s in stock_names:
    stocks[s] = stocks[s].pct_change() * 100.0

stocks = stocks.dropna().drop("Date", axis=1)
stocks

,AAPL,GOOG,IBM,MARA,NVDA
1,5.257744,1.440370,-4.822947,-0.619579,14.159983
2,5.872672,4.478207,1.897465,-11.845387,3.609136
3,-2.258887,-9.846037,-0.978528,-16.265912,0.781988
4,1.174039,-0.284635,-0.427730,30.067568,0.578420
5,-3.828253,-5.539696,-3.295806,-13.376623,8.874133
6,2.944577,5.226636,-0.712268,-4.347826,2.593830
7,-1.675162,-3.201441,-3.232029,-16.457680,-3.871913
8,4.377104,12.581031,-1.402945,53.283302,12.018292
9,3.387098,3.513565,1.293556,-4.406365,4.097185
10,2.901723,-1.942295,4.629256,11.651729,3.726793


In [122]:
stock_return = stocks.mean()
cov_mat = stocks.cov()

# Optimization Model

In [123]:
m = Model("Portfolio")

vars = pd.Series(m.addVars(stock_names), index=stock_names)

portfolio_risk = cov_mat.dot(vars).dot(vars)

m.setObjective(portfolio_risk, GRB.MINIMIZE)

m.addConstr(vars.sum() == 1, "budget")
m.addConstr(stock_return.dot(vars) >= 0.009, "return")

m.optimize()

for v in vars:
    print(f"{v.varName}: {v.x}")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D56)

CPU model: Apple M2 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 2 rows, 5 columns and 10 nonzeros
Model fingerprint: 0x95179706
Model has 15 quadratic objective terms
Coefficient statistics:
  Matrix range     [4e-01, 3e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [4e+00, 6e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e-03, 1e+00]
Presolve time: 0.00s
Presolved: 2 rows, 5 columns, 10 nonzeros
Presolved model has 15 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 4
 AA' NZ     : 1.500e+01
 Factor NZ  : 2.100e+01
 Factor Ops : 9.100e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   5.72937239e+07 -5.72937239e+07  4.00e+03 3.51e-06  1.00e+06     0

# Efficient Frontier

In [124]:
stocks = pd.read_csv("stocks.csv", low_memory=False, parse_dates=["Date"])
stocks["year"] = stocks["Date"].dt.year
stocks["month"] = stocks["Date"].dt.month

for s in stock_names:
    stocks[s] = stocks[s].pct_change() * 100.0

stocks = stocks.dropna().drop("Date", axis=1)
stocks = (
    stocks.groupby(["year", "month"], as_index=False)
    .sum()
    .drop(["year", "month"], axis=1)
)
stocks

,AAPL,GOOG,IBM,MARA,NVDA
0,11.130415,5.918576,-2.925482,-12.464965,17.769120
1,-1.968524,-10.443732,-5.414331,-3.922794,12.828371
2,8.990764,10.950861,1.287838,44.070986,15.970357
3,2.897360,4.163676,-3.598409,28.882894,-0.023770
4,6.658355,15.296720,4.779337,1.713685,38.036822
5,7.050706,-3.420731,1.314804,36.318733,7.788712
6,-6.076060,6.849234,7.656165,15.513621,5.983529
7,4.404447,6.374629,2.596843,-22.922367,9.113947
8,-9.906747,-3.546322,-5.189286,-32.748882,-10.400095
9,3.412034,-0.325277,5.463366,11.511057,4.563411


In [127]:
stock_volatility = stocks.std()
stock_return = stocks.mean()
cov_mat = stocks.cov()

returns = np.linspace(0.2, 0.4, 100)
ret_list = []
risks = []
props = []

In [126]:
for ret in returns:
    m.reset()
    m = Model("Portfolio Optimization")
    m.setParam("OutputFlag", 0)
    vars = pd.Series(m.addVars(stock_names, lb=0), index=stock_names)
    portfolio_risk = cov_mat.dot(vars).dot(vars)
    m.setObjective(portfolio_risk, GRB.MINIMIZE)
    m.addConstr(vars.sum() == 1, "budget")
    m.addConstr(stock_return.dot(vars) == ret, name="return_sim")
    m.update()
    m.optimize()
    risks.append(np.sqrt(m.ObjVal))
    ret_list.append(stock_return.dot(m.x))
    props.append(m.x)

Discarded solution information


AttributeError: Unable to retrieve attribute 'ObjVal'